In [1]:
"""
시대별 K-POP 가사 의미 변화 분석 (Semantic Shift Analysis)
=============================================================
전처리된 멜론 차트 데이터(1996-2025) 활용
"""
import os
import glob
import ast
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


# ============================================================
# Step 1. 전처리된 데이터 로드
# ============================================================

def load_processed_data(lyrics_dir='processed_lyrics'):
    """전처리된 Excel 파일들을 병합"""
    all_files = glob.glob(os.path.join(lyrics_dir, 'Processed_Melon_Chart_*.xlsx'))

    df_list = []
    for file in all_files:
        df = pd.read_excel(file)
        df_list.append(df)

    combined_df = pd.concat(df_list, ignore_index=True)

    print(f"✓ 총 {len(combined_df):,}개 로드")
    print(f"  - 연도 범위: {combined_df['Year'].min()} ~ {combined_df['Year'].max()}")
    print(f"  (중복 제거는 시대별로 수행)")

    return combined_df


def parse_tokens(token_str):
    """문자열 형태의 토큰 리스트를 파싱"""
    if pd.isna(token_str):
        return []
    try:
        # 문자열을 리스트로 변환
        tokens = ast.literal_eval(token_str)
        # 한글 토큰만 필터링 (영어/의성어 제거)
        korean_tokens = [t for t in tokens if is_valid_korean(t)]
        return korean_tokens
    except:
        return []


def is_valid_korean(word):
    """유효한 한글 단어인지 확인"""
    # 한글이 포함되어 있는지 확인
    has_korean = any('가' <= c <= '힣' for c in word)
    if not has_korean:
        return False
    # 의성어/반복어 필터링
    if is_onomatopoeia(word):
        return False
    # 불용어 필터링
    if word in STOPWORDS:
        return False
    return True


def is_onomatopoeia(word):
    """의성어/반복 어구 감지"""
    if len(word) < 2:
        return False
    # 동일 글자 반복 (바바바, 라라라)
    if len(set(word)) == 1:
        return True
    # 2글자 반복 (짠짠, 둠칫둠칫)
    if len(word) >= 4 and len(word) % 2 == 0:
        half = len(word) // 2
        if word[:half] == word[half:]:
            return True
    return False


# 불용어 리스트
STOPWORDS = {
    '하다', '되다', '있다', '없다', '같다', '보다', '주다', '오다', '가다',
    '이다', '아니다', '않다', '못하다', '싶다', '알다', '모르다',
    '그렇다', '어떻다', '이렇다', '저렇다', '이러다', '저러다' , '이번', '저번',
    '나', '너', '너희', '우리', '저', '그', '이것', '그것', '저것',
    '때', '것', '수', '듯', '더', '또', '다시', '너무', '정말', '진짜',
    '아이고', '훨씬', '전혀', '몹시', '매우', '존나', '미처', '완전', '완전히', '일단',
    '일단', '막상' , '그새', '아예', '어쨌든', '이내', '부디', '제발', '결코'
}


# ============================================================
# Step 2. 시대별 데이터 분할 및 모델 학습
# ============================================================

def slice_by_era(df):
    """데이터를 3개 시대로 분할 (시대별 중복 제거)"""
    era_ranges = [
        ('Era1 (1996-2005)', 1996, 2005),
        ('Era2 (2006-2015)', 2006, 2015),
        ('Era3 (2016-2025)', 2016, 2025),
    ]

    eras = {}
    print("\n✓ 시대별 데이터 분할 (시대 내 중복 제거):")

    for name, year_start, year_end in era_ranges:
        era_df = df[(df['Year'] >= year_start) & (df['Year'] <= year_end)]
        original = len(era_df)

        # 시대 내에서 중복 곡 제거
        if 'Song_ID' in era_df.columns:
            era_df = era_df.drop_duplicates(subset=['Song_ID'], keep='first')

        deduped = len(era_df)
        removed = original - deduped
        eras[name] = era_df
        print(f"  - {name}: {original:,} → {deduped:,}곡 (중복 {removed:,}개 제거)")

    return eras


def train_era_model(era_df, era_name, vector_size=100, window=4,
                    min_count=5, min_doc_count=7):
    """시대별 Word2Vec 모델 학습
    - min_count: 전체에서 최소 등장 횟수
    - min_doc_count: 최소 등장 곡 수
    """
    from collections import Counter

    print(f"\n[{era_name}] 모델 학습 시작...")

    # 전처리된 토큰 파싱
    tokenized_data = []
    for tokens_str in era_df['Processed_Tokens']:
        tokens = parse_tokens(tokens_str)
        if len(tokens) >= 3:
            tokenized_data.append(tokens)

    if len(tokenized_data) < 10:
        print(f"  ⚠ 데이터 부족 (토큰화된 곡: {len(tokenized_data)})")
        return None

    # 단어별 등장 곡 수(document frequency) 계산
    doc_freq = Counter()
    for tokens in tokenized_data:
        unique_tokens = set(tokens)  # 곡 내 중복 제거
        doc_freq.update(unique_tokens)

    # min_doc_count 미만인 단어 제거
    rare_words = {w for w, cnt in doc_freq.items() if cnt < min_doc_count}
    filtered_data = [
        [w for w in tokens if w not in rare_words]
        for tokens in tokenized_data
    ]
    filtered_data = [t for t in filtered_data if len(t) >= 3]

    before_vocab = len(set(w for tokens in tokenized_data for w in tokens))
    after_vocab = len(set(w for tokens in filtered_data for w in tokens))
    print(f"  - 곡 수 필터(≥{min_doc_count}곡): {before_vocab:,} → {after_vocab:,} 단어")

    # Word2Vec 학습
    model = Word2Vec(
        sentences=filtered_data,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        workers=4,
        sg=1,  # Skip-gram
        epochs=10
    )

    print(f"  ✓ 학습 완료! (어휘 수: {len(model.wv):,}개)")
    return model


def train_all_models(eras):
    """모든 시대별 모델 학습"""
    models = {}
    for name, data in eras.items():
        model = train_era_model(data, name)
        if model:
            models[name] = model
    return models


# ============================================================
# Step 3. 의미 변화 분석
# ============================================================

def analyze_semantic_shift(models, target_word, topn=15):
    """특정 단어의 시대별 의미 변화 분석"""
    print(f"\n{'='*60}")
    print(f"  '{target_word}'의 시대별 의미 변화 분석")
    print(f"{'='*60}")

    results = {}

    for name, model in models.items():
        try:
            similar_words = model.wv.most_similar(target_word, topn=topn)
            words_only = [w[0] for w in similar_words]

            results[name] = {
                'similar_words': similar_words,
                'words_list': words_only
            }

            print(f"\n[{name}]")
            print(f"  유사 단어: {', '.join(words_only[:10])}")

        except KeyError:
            print(f"\n[{name}] '{target_word}'가 이 시대 데이터에 없습니다.")
            results[name] = None

    return results


def find_word_sources(df, target_word, era_range=None, max_results=10):
    """특정 단어가 포함된 원문 가사 출처 확인"""
    print(f"\n{'='*60}")
    print(f"  '{target_word}' 단어 출처 분석")
    print(f"{'='*60}")

    if era_range:
        filtered_df = df[(df['Year'] >= era_range[0]) & (df['Year'] <= era_range[1])]
        print(f"  분석 범위: {era_range[0]} ~ {era_range[1]}년")
    else:
        filtered_df = df

    matches = []
    for idx, row in filtered_df.iterrows():
        tokens_str = str(row.get('Processed_Tokens', ''))
        if target_word in tokens_str:
            context = extract_context(str(row.get('Lyrics', '')), target_word)
            matches.append({
                'Year': row['Year'],
                'Title': row['Title'],
                'Artist': row['Artist'],
                'Context': context
            })

    # 연도별 빈도
    year_counts = {}
    for m in matches:
        year = m['Year']
        year_counts[year] = year_counts.get(year, 0) + 1

    print(f"\n  총 {len(matches)}곡에서 '{target_word}' 발견")

    if year_counts:
        print(f"\n  [연도별 빈도]")
        for year in sorted(year_counts.keys()):
            bar = '█' * min(year_counts[year], 30)
            print(f"  {year}: {bar} ({year_counts[year]}곡)")

    print(f"\n  [대표 곡 목록]")
    for i, m in enumerate(matches[:max_results]):
        print(f"  {i+1}. [{m['Year']}] {m['Artist']} - {m['Title']}")
        if m['Context']:
            print(f"      \"{m['Context'][:60]}...\"")

    return matches


def extract_context(lyrics, word, window=30):
    """단어 주변 문맥 추출"""
    if not lyrics:
        return ""
    idx = lyrics.find(word)
    if idx == -1:
        return ""
    start = max(0, idx - window)
    end = min(len(lyrics), idx + len(word) + window)
    return lyrics[start:end].replace('\n', ' ')


def deep_dive_word(models, df, target_word, era_name=None):
    """특정 단어 심층 분석"""
    print(f"\n{'#'*60}")
    print(f"  '{target_word}' 심층 분석")
    print(f"{'#'*60}")

    # 유사 단어 분석
    if era_name and era_name in models:
        model = models[era_name]
        try:
            similar = model.wv.most_similar(target_word, topn=15)
            print(f"\n  [{era_name}] '{target_word}'와 유사한 단어:")
            for w, score in similar:
                print(f"    - {w}: {score:.3f}")
        except KeyError:
            print(f"  '{target_word}'가 해당 시대에 없습니다.")
    else:
        for name, model in models.items():
            try:
                similar = model.wv.most_similar(target_word, topn=5)
                words_str = ', '.join([f"{w}({s:.2f})" for w, s in similar])
                print(f"\n  [{name}] {words_str}")
            except KeyError:
                print(f"\n  [{name}] 없음")

    # 출처 확인
    if era_name:
        import re
        years = re.findall(r'\d{4}', era_name)
        if len(years) == 2:
            find_word_sources(df, target_word, era_range=(int(years[0]), int(years[1])), max_results=5)
    else:
        find_word_sources(df, target_word, max_results=10)

# ============================================================
# 메인 실행
# ============================================================

def main():
    print("\n" + "=" * 60)
    print("  K-POP 가사 시대별 의미 변화 분석")
    print("  (Melon Chart 1996-2025, 전처리 데이터)")
    print("=" * 60)

    # 1. 전처리된 데이터 로드
    print("\n[Step 1] 전처리된 데이터 로드")
    df = load_processed_data('processed_lyrics')

    # 2. 시대별 분할
    print("\n[Step 2] 시대별 데이터 분할")
    eras = slice_by_era(df)

    # 3. 모델 학습
    print("\n[Step 3] 시대별 Word2Vec 모델 학습")
    models = train_all_models(eras)

    return models, eras, df

if __name__ == "__main__":
    models, eras, df = main()


  K-POP 가사 시대별 의미 변화 분석
  (Melon Chart 1996-2025, 전처리 데이터)

[Step 1] 전처리된 데이터 로드
✓ 총 31,471개 로드
  - 연도 범위: 1996 ~ 2025
  (중복 제거는 시대별로 수행)

[Step 2] 시대별 데이터 분할

✓ 시대별 데이터 분할 (시대 내 중복 제거):
  - Era1 (1996-2005): 7,771 → 2,224곡 (중복 5,547개 제거)
  - Era2 (2006-2015): 11,792 → 4,799곡 (중복 6,993개 제거)
  - Era3 (2016-2025): 11,908 → 2,307곡 (중복 9,601개 제거)

[Step 3] 시대별 Word2Vec 모델 학습

[Era1 (1996-2005)] 모델 학습 시작...
  - 곡 수 필터(≥7곡): 6,635 → 1,537 단어
  ✓ 학습 완료! (어휘 수: 1,537개)

[Era2 (2006-2015)] 모델 학습 시작...
  - 곡 수 필터(≥7곡): 11,260 → 2,707 단어
  ✓ 학습 완료! (어휘 수: 2,707개)

[Era3 (2016-2025)] 모델 학습 시작...
  - 곡 수 필터(≥7곡): 9,900 → 2,097 단어
  ✓ 학습 완료! (어휘 수: 2,097개)


In [3]:
import numpy as np
import pandas as pd
from scipy.linalg import orthogonal_procrustes

def align_and_compare(model_ref, model_target):
    """
    [엔진] 두 모델을 정렬하고 단어별 코사인 유사도(의미 변화)를 계산
    """
    common_vocab = list(set(model_ref.wv.index_to_key) & set(model_target.wv.index_to_key))
    if not common_vocab:
        return pd.DataFrame(columns=['word', 'shift_score'])

    mat_ref = np.vstack([model_ref.wv[word] for word in common_vocab])
    mat_target = np.vstack([model_target.wv[word] for word in common_vocab])

    # Procrustes 정렬: 두 벡터 공간의 좌표축을 맞춤
    R, _ = orthogonal_procrustes(mat_target, mat_ref)
    aligned_target_mat = mat_target @ R

    shifts = []
    for i, word in enumerate(common_vocab):
        vec_ref = mat_ref[i]
        vec_aligned = aligned_target_mat[i]
        
        # 코사인 유사도 계산
        sim = np.dot(vec_ref, vec_aligned) / (np.linalg.norm(vec_ref) * np.linalg.norm(vec_aligned))
        # 1 - 유사도 = 변화량 (높을수록 의미가 많이 변함)
        shifts.append({'word': word, 'shift_score': 1 - sim})

    return pd.DataFrame(shifts)

def get_multi_era_top_shifts(models, top_n=30):
    """
    [분석기] Era 1->2, Era 2->3의 변화를 순차적으로 비교하여 누적 변화량 계산
    """
    era_names = list(models.keys())
    if len(era_names) < 3:
        print("⚠ 분석을 위해 최소 3개의 시대별 모델이 필요합니다.")
        return []

    # 1. 시기별 변화량 계산 (1-2 구간, 2-3 구간)
    print(f"진행 중: {era_names[0]} vs {era_names[1]}...")
    shift_1_2 = align_and_compare(models[era_names[0]], models[era_names[1]])
    
    print(f"진행 중: {era_names[1]} vs {era_names[2]}...")
    shift_2_3 = align_and_compare(models[era_names[1]], models[era_names[2]])

    # 2. 데이터 병합 및 누적 점수 계산
    total_shift = pd.merge(shift_1_2, shift_2_3, on='word', suffixes=('_12', '_23'))
    total_shift['total_shift_score'] = total_shift['shift_score_12'] + total_shift['shift_score_23']
    
    # 3. 정렬 및 결과 출력
    total_shift = total_shift.sort_values('total_shift_score', ascending=False)
    
    print(f"\n" + "="*60)
    print(f"  ★ K-POP 시대별 순차적 의미 변화 TOP {top_n} ★")
    print(f"  (비교 구간: {era_names[0]}->{era_names[1]} 및 {era_names[1]}->{era_names[2]})")
    print("="*60)
    
    for i, row in enumerate(total_shift.head(top_n).itertuples(), 1):
        print(f"{i:2d}. {row.word:<10} (합계 변화량: {row.total_shift_score:.4f})")
        print(f"    [{era_names[0]}->{era_names[1]}: {row.shift_score_12:.3f}] -> [{era_names[1]}->{era_names[2]}: {row.shift_score_23:.3f}]")
        print("-" * 55)
    
    return total_shift

# ---------------------------------------------------------
# 실행 (models 객체가 준비된 상태에서)
# ---------------------------------------------------------
result_df = get_multi_era_top_shifts(models)

진행 중: Era1 (1996-2005) vs Era2 (2006-2015)...
진행 중: Era2 (2006-2015) vs Era3 (2016-2025)...

  ★ K-POP 시대별 순차적 의미 변화 TOP 30 ★
  (비교 구간: Era1 (1996-2005)->Era2 (2006-2015) 및 Era2 (2006-2015)->Era3 (2016-2025))
 1. 어떡         (합계 변화량: 1.3001)
    [Era1 (1996-2005)->Era2 (2006-2015): 0.660] -> [Era2 (2006-2015)->Era3 (2016-2025): 0.640]
-------------------------------------------------------
 2. 널          (합계 변화량: 1.2890)
    [Era1 (1996-2005)->Era2 (2006-2015): 0.712] -> [Era2 (2006-2015)->Era3 (2016-2025): 0.577]
-------------------------------------------------------
 3. 어설프다       (합계 변화량: 1.2369)
    [Era1 (1996-2005)->Era2 (2006-2015): 0.582] -> [Era2 (2006-2015)->Era3 (2016-2025): 0.655]
-------------------------------------------------------
 4. 세우다        (합계 변화량: 1.2361)
    [Era1 (1996-2005)->Era2 (2006-2015): 0.639] -> [Era2 (2006-2015)->Era3 (2016-2025): 0.597]
-------------------------------------------------------
 5. 거부         (합계 변화량: 1.2196)
    [Era1 (1996-2005)->Era2

In [4]:
import numpy as np
import pandas as pd
import os
from scipy.linalg import orthogonal_procrustes

def align_and_compare(model_ref, model_target):
    """
    [엔진] 두 모델을 정렬하고 단어별 코사인 유사도(의미 변화)를 계산
    """
    common_vocab = list(set(model_ref.wv.index_to_key) & set(model_target.wv.index_to_key))
    if not common_vocab:
        return pd.DataFrame(columns=['word', 'shift_score'])

    mat_ref = np.vstack([model_ref.wv[word] for word in common_vocab])
    mat_target = np.vstack([model_target.wv[word] for word in common_vocab])

    # Procrustes 정렬
    R, _ = orthogonal_procrustes(mat_target, mat_ref)
    aligned_target_mat = mat_target @ R

    shifts = []
    for i, word in enumerate(common_vocab):
        vec_ref = mat_ref[i]
        vec_aligned = aligned_target_mat[i]
        sim = np.dot(vec_ref, vec_aligned) / (np.linalg.norm(vec_ref) * np.linalg.norm(vec_aligned))
        shifts.append({'word': word, 'shift_score': 1 - sim})

    return pd.DataFrame(shifts)

def get_sequential_shift_and_save(models, top_n=30, output_file='output/sequential_semantic_shifts.csv'):
    """
    [분석기] Era 1->2, Era 2->3 순차적 변화량 계산 및 CSV 저장
    """
    era_names = list(models.keys())
    if len(era_names) < 3:
        print("⚠ 3개 이상의 시대 모델이 필요합니다.")
        return None

    # 1. 구간별 변화량 계산 (1-2, 2-3)
    print(f"[{era_names[0]} vs {era_names[1]}] 분석 중...")
    s12 = align_and_compare(models[era_names[0]], models[era_names[1]])
    
    print(f"[{era_names[1]} vs {era_names[2]}] 분석 중...")
    s23 = align_and_compare(models[era_names[1]], models[era_names[2]])

    # 2. 데이터 병합 및 누적 점수 계산
    total = pd.merge(s12, s23, on='word', suffixes=('_12', '_23'))
    total['total_shift_score'] = total['shift_score_12'] + total['shift_score_23']
    
    # 3. 정렬 및 순위 부여
    total = total.sort_values('total_shift_score', ascending=False).reset_index(drop=True)
    total.index = total.index + 1 # 순위를 1부터 시작하게 설정

    # 4. CSV 저장
    os.makedirs('output', exist_ok=True)
    total.to_csv(output_file, encoding='utf-8-sig', index_label='순위')
    
    print(f"\n" + "="*60)
    print(f"✓ 분석 결과 저장 완료: {output_file}")
    print(f"★ Era 1->2->3 누적 변화 TOP {top_n} ★")
    print("="*60)
    
    for i, row in enumerate(total.head(top_n).itertuples(), 1):
        print(f"{i:2d}. {row.word:<10} (합계: {row.total_shift_score:.4f})")
        print(f"    [E1->E2: {row.shift_score_12:.3f}] [E2->E3: {row.shift_score_23:.3f}]")
    
    return total

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
result_df = get_sequential_shift_and_save(models)

[Era1 (1996-2005) vs Era2 (2006-2015)] 분석 중...
[Era2 (2006-2015) vs Era3 (2016-2025)] 분석 중...

✓ 분석 결과 저장 완료: output/sequential_semantic_shifts.csv
★ Era 1->2->3 누적 변화 TOP 30 ★
 1. 어떡         (합계: 1.3001)
    [E1->E2: 0.660] [E2->E3: 0.640]
 2. 널          (합계: 1.2890)
    [E1->E2: 0.712] [E2->E3: 0.577]
 3. 어설프다       (합계: 1.2369)
    [E1->E2: 0.582] [E2->E3: 0.655]
 4. 세우다        (합계: 1.2361)
    [E1->E2: 0.639] [E2->E3: 0.597]
 5. 거부         (합계: 1.2196)
    [E1->E2: 0.496] [E2->E3: 0.723]
 6. 분명히        (합계: 1.2174)
    [E1->E2: 0.573] [E2->E3: 0.645]
 7. 충분히        (합계: 1.2089)
    [E1->E2: 0.628] [E2->E3: 0.581]
 8. 피하다        (합계: 1.1994)
    [E1->E2: 0.664] [E2->E3: 0.535]
 9. 자연         (합계: 1.1909)
    [E1->E2: 0.597] [E2->E3: 0.594]
10. 아니         (합계: 1.1899)
    [E1->E2: 0.618] [E2->E3: 0.572]
11. 철없다        (합계: 1.1891)
    [E1->E2: 0.593] [E2->E3: 0.596]
12. 고프다        (합계: 1.1793)
    [E1->E2: 0.546] [E2->E3: 0.633]
13. 가르치다       (합계: 1.1752)
    [E1->E2: 0.570] [E2->E3